In [1]:
import cPickle as pkl
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped
import cPickle as pkl
import sys
sys.setrecursionlimit(15000)
import pandas as pd
import cspython.analysis as a

In [2]:
with open('../cspython/NRG_2016-10-01_to_2017-10-13.pkl', 'rb') as f:
    d = pkl.load(f)
    
big_data = process_scrapped(d)
overview, big_data = big_data

In [65]:
cur_roster = ['RIKO', 'FugLy', 'Brehze', 'AnJ', 'CeRq', 'daps']  #NRG
"""
cur_roster = ['gob b', 'LEGIJA', 'tabseN', 'nex', 'keev', 'kakafu'] #Big
cur_roster = ['Skadoodle', 'Stewie2k', 'autimatic', 'RUSH', 'tarik', 'valens'] #Cloud9
cur_roster = ['reltuC', 'koosta', 'nahtE', 'FNS', 'Rickeh', 'Ryu'] #Counter Logic Gaming
cur_roster = ['chrisj', 'oskar', 'ropz', 'suNny', 'STYKO', 'lmbt'] #mousesports
cur_roster = ['AZR', 'jks', 'USTILO', 'Nifty', 'NAF', 'kassad']  #Renegades
cur_roster = ['dzt', 'land1n', 'tatazin', 'shz', 's1', 'bLecker'] #Tempo Storm
cur_roster = ['ANGE1', 'Zero', 'DeadFox', 'woxic', 'ISSAA', 'Johnta']  #HellRaisers
"""

"\ncur_roster = ['gob b', 'LEGIJA', 'tabseN', 'nex', 'keev', 'kakafu'] #Big\ncur_roster = ['Skadoodle', 'Stewie2k', 'autimatic', 'RUSH', 'tarik', 'valens'] #Cloud9\ncur_roster = ['reltuC', 'koosta', 'nahtE', 'FNS', 'Rickeh', 'Ryu'] #Counter Logic Gaming\ncur_roster = ['chrisj', 'oskar', 'ropz', 'suNny', 'STYKO', 'lmbt'] #mousesports\ncur_roster = ['AZR', 'jks', 'USTILO', 'Nifty', 'NAF', 'kassad']  #Renegades\ncur_roster = ['dzt', 'land1n', 'tatazin', 'shz', 's1', 'bLecker'] #Tempo Storm\ncur_roster = ['ANGE1', 'Zero', 'DeadFox', 'woxic', 'ISSAA', 'Johnta']  #HellRaisers\n"

In [66]:
def map_roster_analysis(big_data, team, cur_roster):
    roster_matches = a.roster_match(big_data, team, cur_roster)
    match_overview = a.create_team_match_overview(big_data, team)

    match_overview.loc[:,'roster_matches'] = match_overview.series_id.map(roster_matches)
    match_overview.loc[:,'roster_weight'] = (match_overview.loc[:,'roster_matches']/5).astype(float)

    match_overview.loc[:,'t_win_ratio'] = (match_overview.loc[:,'t_wins']/match_overview.loc[:,'t_rounds_played']).astype(float)
    match_overview.loc[:,'ct_win_ratio'] = (match_overview.loc[:,'ct_wins']/match_overview.loc[:,'ct_rounds_played']).astype(float)
    match_overview.loc[:,'t_win_ratio_weighted'] = (match_overview.loc[:,'t_wins']/match_overview.loc[:,'t_rounds_played']).astype(float)*match_overview.roster_weight
    match_overview.loc[:,'ct_win_ratio_weighted'] = (match_overview.loc[:,'ct_wins']/match_overview.loc[:,'ct_rounds_played']).astype(float)*match_overview.roster_weight


    for col in ['t_wins', 'ct_wins', 't_rounds_played', 'ct_rounds_played']:
        match_overview.loc[:, col] = pd.to_numeric(match_overview.loc[:, col])
        
    round_count = match_overview.groupby('map').sum()[['t_rounds_played', 'ct_rounds_played']]
    ratios = match_overview.groupby('map').mean().loc[:,['t_win_ratio', 'ct_win_ratio']]
    t_weighted = match_overview.groupby('map').sum().loc[:,'t_win_ratio_weighted']/match_overview.groupby('map').sum().loc[:,'roster_weight']
    ct_weighted = match_overview.groupby('map').sum().loc[:,'ct_win_ratio_weighted']/match_overview.groupby('map').sum().loc[:,'roster_weight']

    results = pd.concat([ratios, t_weighted, ct_weighted, round_count], axis=1)
    results.columns = ['t_win_ratio', 'ct_win_ratio', 
                       'roster_weighted_t_win_ratio', 
                       'roster_weighted_ct_win_ratio', 
                       't_rounds_played',
                       'ct_rounds_played']
    
    return results

In [68]:
map_roster_analysis(big_data, 'NRG', cur_roster).to_csv('sample_nrg_map_data.csv')